In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df1=pd.read_csv('Test_Claim.csv')
df2=pd.read_csv('Test_Demographics.csv')
df3=pd.read_csv('Test_Policy.csv')

In [3]:
df3.shape

(8912, 10)

In [4]:
df4=pd.read_csv('Test_Vehicle.csv')

## Filtering VehicleAttribute 

In [5]:
#Creating an VechileID column
v_id=df4.loc[df4.VehicleAttribute=='VehicleID']
v_id['VehicleID']=v_id['VehicleAttributeDetails']
v_id.drop(['VehicleAttribute','VehicleAttributeDetails'],axis=1,inplace=True)

In [6]:
#creating VehicleModel column
vm=df4.loc[df4.VehicleAttribute=='VehicleModel']
vm['VehicleModel']=vm['VehicleAttributeDetails']
vm.drop(['VehicleAttribute','VehicleAttributeDetails'],axis=1,inplace=True)

In [7]:
#creating VehicleMake column
v_make=df4.loc[df4.VehicleAttribute=='VehicleMake']
v_make['VehicleMake']=v_make['VehicleAttributeDetails']
v_make.drop(['VehicleAttribute','VehicleAttributeDetails'],axis=1,inplace=True)

In [8]:
#creating VehicleYOM column
v_yom=df4.loc[df4.VehicleAttribute=='VehicleYOM']
v_yom['VehicleYOM']=v_yom['VehicleAttributeDetails']
v_yom.drop(['VehicleAttribute','VehicleAttributeDetails'],axis=1,inplace=True)

d1=v_id.merge(vm)
d2=v_make.merge(v_yom)

df5=d1.merge(d2)

In [9]:
m1=df1.merge(df2)
m2=df3.merge(df5)

df_new=m1.merge(m2)

In [10]:
df_new.shape

(8912, 41)

# Replacing NaN 

In [11]:
df_new.replace('?',np.nan,inplace=True)
df_new.replace(-1.0,np.nan,inplace=True)
df_new.replace(-5.0,np.nan,inplace=True)
df_new.replace('MISSINGVAL',np.nan,inplace=True)
df_new.replace('MISSINGVALUE',np.nan,inplace=True)
df_new.replace('MISSEDDATA',np.nan,inplace=True)
df_new.replace('???',np.nan,inplace=True)

In [12]:
df_new.to_csv('final_clean.csv',index=False)

In [13]:
df_new.isnull().sum()

CustomerID                       0
DateOfIncident                   0
TypeOfIncident                   0
TypeOfCollission              1763
SeverityOfIncident               0
AuthoritiesContacted             0
IncidentState                    0
IncidentCity                     0
IncidentAddress                  0
IncidentTime                     7
NumberOfVehicles                 0
PropertyDamage                3199
BodilyInjuries                   0
Witnesses                       12
PoliceReport                  3014
AmountOfTotalClaim               8
AmountOfInjuryClaim              0
AmountOfPropertyClaim            0
AmountOfVehicleDamage            0
InsuredAge                       0
InsuredZipCode                   0
InsuredGender                    8
InsuredEducationLevel            0
InsuredOccupation                0
InsuredHobbies                   0
CapitalGains                     0
CapitalLoss                      0
Country                          4
InsurancePolicyNumbe

In [14]:
df_new['TypeOfCollission'].unique()

array(['Front Collision', 'Rear Collision', nan, 'Side Collision'],
      dtype=object)

# Missing value Treatment


## i)Missing value treatment for 'PoliceReport' column

In [15]:
# We are going to predict police report column . Train claim dataset having this column.
#so we are going to take this dataset and make prediction  for the particular column
p1=pd.read_csv('Test_claim.csv')
p1.replace('?',np.nan,inplace=True)

In [16]:
cat=p1.select_dtypes(include=['object'])
cat.columns

Index(['CustomerID', 'DateOfIncident', 'TypeOfIncident', 'TypeOfCollission',
       'SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState',
       'IncidentCity', 'IncidentAddress', 'PropertyDamage', 'Witnesses',
       'PoliceReport', 'AmountOfTotalClaim'],
      dtype='object')

In [17]:
#we take values from police report column which having not null values
#We take this value for training purpose
not_null=p1.loc[p1['PoliceReport'].notnull()]
not_null.shape

(5898, 19)

In [18]:
#copying original file for other process
n1=not_null.copy()

In [19]:
# Converting categorical to numerical 
cat_col=['DateOfIncident', 'TypeOfIncident', 'TypeOfCollission',
       'SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState',
       'IncidentCity', 'IncidentAddress', 'PropertyDamage', 'Witnesses',
       'PoliceReport', 'AmountOfTotalClaim']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in cat_col:
    n1[col]= le.fit_transform(n1[col])

In [20]:
#Droping 'polciereport' column and assign to new valriable
not_null=not_null.drop(['PoliceReport'],axis=1)

#Assigning the numerical value for 'policereport' column
not_null['PoliceReport']=n1['PoliceReport']

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [22]:
x=n1.drop(['PoliceReport','CustomerID'],axis=1)
y=n1['PoliceReport']

In [23]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.1,random_state=0)

In [24]:
logi=LogisticRegression()
logi.fit(X_train,Y_train)

LogisticRegression()

In [25]:
pred_train=logi.predict(X_train)
pred_test=logi.predict(X_test)

In [26]:
print(accuracy_score(Y_train,pred_train))
print(accuracy_score(Y_test,pred_test))

0.5276940467219292
0.5525423728813559


In [27]:
from sklearn.tree import DecisionTreeClassifier

In [28]:
clf=DecisionTreeClassifier(criterion='gini')
clf.fit(X_train,Y_train)
pred_train=clf.predict(X_train)
pred_test=clf.predict(X_test)

In [29]:
print(accuracy_score(Y_train,pred_train))
print(accuracy_score(Y_test,pred_test))

1.0
0.7050847457627119


In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
classi=RandomForestClassifier(n_estimators=40,criterion='entropy')
classi.fit(X_train,Y_train)
pred_train=classi.predict(X_train)
pred_test=classi.predict(X_test)


In [32]:
print(accuracy_score(Y_train,pred_train))
print(accuracy_score(Y_test,pred_test))

0.9998116051243406
0.7745762711864407


In [33]:
null=p1.loc[p1['PoliceReport'].isnull()]
null.shape

(3014, 19)

In [34]:
x1=null.drop(['PoliceReport','CustomerID'],axis=1)

In [35]:
cat_col=['DateOfIncident', 'TypeOfIncident', 'TypeOfCollission',
       'SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState',
       'IncidentCity', 'IncidentAddress', 'PropertyDamage', 'Witnesses','AmountOfTotalClaim']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in cat_col:
    x1[col]= le.fit_transform(x1[col])

In [36]:
prediction=classi.predict(x1)

In [37]:
null.drop(['PoliceReport'],axis=1,inplace=True)
null['PoliceReport']=prediction

n_final=pd.concat([not_null,null])
n_final.shape

(8912, 19)

In [38]:
#Drop old column and add new values to that column
df_new.drop(['PoliceReport'],axis=1,inplace=True)
df_new['PoliceReport']=n_final['PoliceReport']

## ii)Missing value treatment for 'TypeOfCollission' column

In [39]:
#we store rows based on Type of collission column which are not having null values
c1=p1.drop(['PoliceReport'],axis=1)
c1['PoliceReport']=n_final['PoliceReport']

In [40]:
coli=c1.loc[c1['TypeOfCollission'].notnull()]
coli_c=coli.copy()

In [41]:
cat_col=['DateOfIncident', 'TypeOfIncident', 'TypeOfCollission',
       'SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState',
       'IncidentCity', 'IncidentAddress', 'PropertyDamage', 'Witnesses', 'AmountOfTotalClaim']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in cat_col:
    coli_c[col]= le.fit_transform(coli_c[col])

In [42]:
#Droping 'TypeOfCollission' column and assign to new valriable
coli=coli.drop(['TypeOfCollission'],axis=1)
coli['TypeOfCollission']=coli_c['TypeOfCollission']

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [44]:
x=coli_c.drop(['TypeOfCollission','CustomerID'],axis=1)
y=coli_c['TypeOfCollission']

In [45]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.1,random_state=0)

In [46]:
logi1=LogisticRegression(max_iter=1000)
logi1.fit(X_train,Y_train)
pred_train=logi.predict(X_train)
pred_test=logi.predict(X_test)

In [47]:
print(accuracy_score(Y_train,pred_train))
print(accuracy_score(Y_test,pred_test))

0.3030774013055642
0.3090909090909091


In [48]:
clf1=DecisionTreeClassifier(criterion='gini')
clf1.fit(X_train,Y_train)
pred_train=clf1.predict(X_train)
pred_test=clf1.predict(X_test)

In [49]:
print(accuracy_score(Y_train,pred_train))
print(accuracy_score(Y_test,pred_test))

1.0
0.5300699300699301


In [50]:
classifier=RandomForestClassifier(n_estimators=30,criterion='entropy')
classifier.fit(X_train,Y_train)
pred_train=classifier.predict(X_train)
pred_test=classifier.predict(X_test)

In [51]:
print(accuracy_score(Y_train,pred_train))
print(accuracy_score(Y_test,pred_test))

0.9993783027665527
0.6559440559440559


In [52]:
co=c1.loc[c1['TypeOfCollission'].isnull()]
co2=co.drop(['TypeOfCollission','CustomerID'],axis=1)

In [53]:
cat_col=['DateOfIncident', 'TypeOfIncident','SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState',
       'IncidentCity', 'IncidentAddress', 'PropertyDamage', 'Witnesses','AmountOfTotalClaim']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in cat_col:
    co2[col]= le.fit_transform(co2[col])


In [54]:
prediction1=classifier.predict(co2)
co.drop(['TypeOfCollission'],axis=1,inplace=True)
co['TypeOfCollission']=prediction1

In [55]:
n_final1=pd.concat([coli,co])
n_final1.shape

(8912, 19)

In [56]:
#Drop old column and add new values to that column
df_new.drop(['TypeOfCollission'],axis=1,inplace=True)
df_new['TypeOfCollission']=n_final1['TypeOfCollission']

## iii) Missing value treatment for 'PropertyDamage'

In [57]:
#we store rows based on policereport column which are not having null values
pr1=p1.drop(['PoliceReport','TypeOfCollission'],axis=1)
pr1['PoliceReport']=n_final['PoliceReport']
pr1['TypeOfCollission']=n_final1['TypeOfCollission']

In [58]:
pr=pr1.loc[pr1['PropertyDamage'].notnull()]
pr.shape
pro=pr.copy()

In [59]:
cat_col=['DateOfIncident', 'TypeOfIncident',
       'SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState',
       'IncidentCity', 'IncidentAddress', 'PropertyDamage', 'Witnesses', 'AmountOfTotalClaim']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in cat_col:
    pro[col]= le.fit_transform(pro[col])

In [60]:
#Droping 'TypeOfCollission' column and assign to new valriable
pr=pr.drop(['PropertyDamage'],axis=1)
pr['PropertyDamage']=pro['PropertyDamage']

In [61]:
x=pro.drop(['PropertyDamage','CustomerID'],axis=1)
y=pro['PropertyDamage']

In [62]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.1,random_state=0)

In [63]:
logi2=LogisticRegression()
logi2.fit(X_train,Y_train)
pred_train=logi2.predict(X_train)
pred_test=logi2.predict(X_test)

In [64]:
print(accuracy_score(Y_train,pred_train))
print(accuracy_score(Y_test,pred_test))

0.5209103287298191
0.506993006993007


In [65]:
clf2=DecisionTreeClassifier(criterion='gini')
clf2.fit(X_train,Y_train)
pred_train=clf2.predict(X_train)
pred_test=clf2.predict(X_test)

In [66]:
print(accuracy_score(Y_train,pred_train))
print(accuracy_score(Y_test,pred_test))

1.0
0.7185314685314685


In [67]:
classifier1=RandomForestClassifier(n_estimators=40,criterion='entropy')
classifier1.fit(X_train,Y_train)
pred_train=classifier1.predict(X_train)
pred_test=classifier1.predict(X_test)

In [68]:
print(accuracy_score(Y_train,pred_train))
print(accuracy_score(Y_test,pred_test))

1.0
0.8146853146853147


In [69]:
prnull=pr1.loc[pr1['PropertyDamage'].isnull()]
prnull.shape
prn1=prnull.drop(['PropertyDamage','CustomerID'],axis=1)

In [70]:
cat_col=['DateOfIncident', 'TypeOfIncident','SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState',
       'IncidentCity', 'IncidentAddress', 'Witnesses','AmountOfTotalClaim']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in cat_col:
    prn1[col]= le.fit_transform(prn1[col])

In [71]:
prediction2=classifier1.predict(prn1)

In [72]:
prnull.drop(['PropertyDamage'],axis=1,inplace=True)
prnull['PropertyDamage']=prediction2

In [73]:
n_final2=pd.concat([pr,prnull])

In [74]:
#Drop old column and add new values to that column
df_new.drop(['PropertyDamage'],axis=1,inplace=True)
df_new['PropertyDamage']=n_final2['PropertyDamage']
df_new.shape

(8912, 41)

In [75]:
df_new.isnull().sum()

CustomerID                     0
DateOfIncident                 0
TypeOfIncident                 0
SeverityOfIncident             0
AuthoritiesContacted           0
IncidentState                  0
IncidentCity                   0
IncidentAddress                0
IncidentTime                   7
NumberOfVehicles               0
BodilyInjuries                 0
Witnesses                     12
AmountOfTotalClaim             8
AmountOfInjuryClaim            0
AmountOfPropertyClaim          0
AmountOfVehicleDamage          0
InsuredAge                     0
InsuredZipCode                 0
InsuredGender                  8
InsuredEducationLevel          0
InsuredOccupation              0
InsuredHobbies                 0
CapitalGains                   0
CapitalLoss                    0
Country                        4
InsurancePolicyNumber          0
CustomerLoyaltyPeriod          0
DateOfPolicyCoverage           0
InsurancePolicyState           0
Policy_CombinedSingleLimit     0
Policy_Ded

In [76]:
df_new['PolicyAnnualPremium'].fillna(df_new['PolicyAnnualPremium'].median(),inplace=True)

df_new['InsuredGender'].fillna(df_new['InsuredGender'].mode()[0],inplace=True)

df_new['Witnesses'].fillna(df_new['Witnesses'].mode()[0],inplace=True)

df_new['VehicleMake'].fillna(df_new['VehicleMake'].mode()[0],inplace=True)

#Changing AmountOfTotalClaim datatype into int
df_new['AmountOfTotalClaim']=df_new['AmountOfTotalClaim'].fillna(-2)

df_new['AmountOfTotalClaim'] = df_new['AmountOfTotalClaim'].astype('int64')

df_new['AmountOfTotalClaim'].replace(-2,df_new['AmountOfTotalClaim'].median(),inplace=True)

df_new['IncidentTime'].fillna(df_new['IncidentTime'].median(),inplace=True)

df_new['Country'].fillna(df_new['Country'].mode(),inplace=True)


## Dropping columns 

In [77]:
df_new1=df_new.drop(['IncidentState','VehicleID','IncidentAddress','IncidentCity','InsuredZipCode','Country',
            'InsurancePolicyNumber','DateOfIncident','InsuredRelationship','InsurancePolicyState',
                    'DateOfPolicyCoverage'],axis=1)

df_new1.shape

(8912, 30)

In [78]:
df_new1.isnull().sum()

CustomerID                    0
TypeOfIncident                0
SeverityOfIncident            0
AuthoritiesContacted          0
IncidentTime                  0
NumberOfVehicles              0
BodilyInjuries                0
Witnesses                     0
AmountOfTotalClaim            0
AmountOfInjuryClaim           0
AmountOfPropertyClaim         0
AmountOfVehicleDamage         0
InsuredAge                    0
InsuredGender                 0
InsuredEducationLevel         0
InsuredOccupation             0
InsuredHobbies                0
CapitalGains                  0
CapitalLoss                   0
CustomerLoyaltyPeriod         0
Policy_CombinedSingleLimit    0
Policy_Deductible             0
PolicyAnnualPremium           0
UmbrellaLimit                 0
VehicleModel                  0
VehicleMake                   0
VehicleYOM                    0
PoliceReport                  0
TypeOfCollission              0
PropertyDamage                0
dtype: int64

# Converting categorical to numerical

In [79]:
df_new1['Witnesses']=df_new1['Witnesses'].astype('int')

In [80]:
ob1=df_new1.select_dtypes(include=['object'])
ob1.columns

Index(['CustomerID', 'TypeOfIncident', 'SeverityOfIncident',
       'AuthoritiesContacted', 'InsuredGender', 'InsuredEducationLevel',
       'InsuredOccupation', 'InsuredHobbies', 'Policy_CombinedSingleLimit',
       'VehicleModel', 'VehicleMake', 'VehicleYOM'],
      dtype='object')

In [81]:
cat_col=['TypeOfIncident', 'SeverityOfIncident',
       'AuthoritiesContacted', 'InsuredGender',
       'InsuredEducationLevel', 'InsuredOccupation','InsuredHobbies',
        'Policy_CombinedSingleLimit', 'VehicleModel', 'VehicleMake',
       'VehicleYOM']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in cat_col:
    df_new1[col]= le.fit_transform(df_new1[col])

In [82]:
df_new1.shape

(8912, 30)

In [83]:
df_new1.to_csv('cleanedtes7.csv',index=False)

In [84]:
df_new1.shape

(8912, 30)